In [53]:
import collections
import functools
import glob
import pickle
import itertools
import json
import os
import random
import sys
import numpy as np
import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# If domainbed is a custom module, you might need to adjust its imports or setup
from domainbed import datasets
from domainbed import algorithms
from domainbed.lib import misc, reporting
from domainbed import model_selection
from domainbed.lib.query import Q
import warnings


In [59]:
def count_subdirectories_and_empty_ones(directory):
    
    num_subdirectories = 0
    num_empty_subdirectories = 0

    # Walk through all directories in the given directory
    for root, dirs, files in os.walk(directory):
        # Iterate over each directory in the current root
        for d in dirs:
            num_subdirectories += 1
            subdirectory_path = os.path.join(root, d)
            # Check if the directory is empty
            if not os.listdir(subdirectory_path):
                num_empty_subdirectories += 1

    return num_subdirectories, num_empty_subdirectories


# Specify the directory you want to inspect
directory_path = 'results_resnet_new'

# Get the count of subdirectories and empty subdirectories
subdirectories, empty_subdirectories = count_subdirectories_and_empty_ones(directory_path)
print(f"Total subdirectories: {subdirectories}")
print(f"Empty subdirectories: {empty_subdirectories}")

Total subdirectories: 50
Empty subdirectories: 0


In [55]:
def format_mean(data, latex):
    """Given a list of datapoints, return a string describing their mean and
    standard error"""
    if len(data) == 0:
        return None, None, "X"
    mean = 100 * np.mean(list(data))
    err = 100 * np.std(list(data) / np.sqrt(len(data)))
    if latex:
        return mean, err, "{:.1f} $\\pm$ {:.1f}".format(mean, err)
    else:
        return mean, err, "{:.1f} +/- {:.1f}".format(mean, err)

def print_table(table, header_text, row_labels, col_labels, colwidth=10,
    latex=True):
    """Pretty-print a 2D array of data, optionally with row/col labels"""
    print("")

    if latex:
        num_cols = len(table[0])
        print("\\begin{center}")
        print("\\adjustbox{max width=\\textwidth}{%")
        print("\\begin{tabular}{l" + "c" * num_cols + "}")
        print("\\toprule")
    else:
        print("--------", header_text)

    for row, label in zip(table, row_labels):
        row.insert(0, label)

    if latex:
        col_labels = ["\\textbf{" + str(col_label).replace("%", "\\%") + "}"
            for col_label in col_labels]
    table.insert(0, col_labels)

    for r, row in enumerate(table):
        misc.print_row(row, colwidth=colwidth, latex=latex)
        if latex and r == 0:
            print("\\midrule")
    if latex:
        print("\\bottomrule")
        print("\\end{tabular}}")
        print("\\end{center}")

def print_results_tables(records, selection_method, latex):
    """Given all records, print a results table for each dataset."""
    grouped_records = reporting.get_grouped_records(records).map(lambda group:
        { **group, "sweep_acc": selection_method.sweep_acc(group["records"]) }
    ).filter(lambda g: g["sweep_acc"] is not None)

    # read algorithm names and sort (predefined order)
    alg_names = Q(records).select("args.algorithm").unique()
    alg_names = ([n for n in algorithms.ALGORITHMS if n in alg_names] +
        [n for n in alg_names if n not in algorithms.ALGORITHMS])

    # read dataset names and sort (lexicographic order)
    dataset_names = Q(records).select("args.dataset").unique().sorted()
    dataset_names = [d for d in datasets.DATASETS if d in dataset_names]

    for dataset in dataset_names:
        if latex:
            print()
            print("\\subsubsection{{{}}}".format(dataset))
        test_envs = range(datasets.num_environments(dataset))
        # breakpoint()
        table = [[None for _ in [*test_envs, "Avg"]] for _ in alg_names]
        for i, algorithm in enumerate(alg_names):
            means = []
            for j, test_env in enumerate(test_envs):
                trial_accs = (grouped_records
                    .filter_equals(
                        "dataset, algorithm, test_env",
                        (dataset, algorithm, test_env)
                    ).select("sweep_acc"))
                mean, err, table[i][j] = format_mean(trial_accs, latex)
                means.append(mean)
            if None in means:
                table[i][-1] = "X"
            else:
                table[i][-1] = "{:.1f}".format(sum(means) / len(means))

        col_labels = [
            "Algorithm",
            *datasets.get_dataset_class(dataset).ENVIRONMENTS,
            "Avg"
        ]
        header_text = (f"Dataset: {dataset}, "
            f"model selection method: {selection_method.name}")
        print_table(table, header_text, alg_names, list(col_labels),
            colwidth=20, latex=latex)

    # Print an "averages" table
    if latex:
        print()
        print("\\subsubsection{Averages}")

    table = [[None for _ in [*dataset_names, "Avg"]] for _ in alg_names]
    for i, algorithm in enumerate(alg_names):
        means = []
        for j, dataset in enumerate(dataset_names):
            trial_averages = (grouped_records
                .filter_equals("algorithm, dataset", (algorithm, dataset))
                .group("trial_seed")
                .map(lambda trial_seed, group:
                    group.select("sweep_acc").mean()
                )
            )
            mean, err, table[i][j] = format_mean(trial_averages, latex)
            means.append(mean)
        if None in means:
            table[i][-1] = "X"
        else:
            table[i][-1] = "{:.1f}".format(sum(means) / len(means))

    col_labels = ["Algorithm", *dataset_names, "Avg"]
    header_text = f"Averages, model selection method: {selection_method.name}"
    print_table(table, header_text, alg_names, col_labels, colwidth=25,
        latex=latex)

In [56]:
input_dir = "./results_resnet_new"
latex = False
results_file = "results.tex" if latex else "results.txt"

In [57]:
records = reporting.load_records(input_dir)
selection_methods = [
    model_selection.IIDAccuracySelectionMethod,
    model_selection.OracleSelectionMethod
]

for selection_method in selection_methods:
    print_results_tables(records, selection_method, latex)



-------- Dataset: RotatedMNIST, model selection method: training-domain validation set
Algorithm             0                     15                    30                    45                    60                    75                    Avg                  
HessianAlignment      94.9 +/- 0.0          98.1 +/- 0.0          98.7 +/- 0.0          98.6 +/- 0.0          98.8 +/- 0.0          96.1 +/- 0.0          97.5                 

-------- Dataset: VLCS, model selection method: training-domain validation set
Algorithm             C                     L                     S                     V                     Avg                  
HessianAlignment      98.4 +/- 0.0          65.3 +/- 0.0          71.6 +/- 0.0          72.5 +/- 0.0          77.0                 

-------- Averages, model selection method: training-domain validation set
Algorithm                  RotatedMNIST               VLCS                       Avg                       
HessianAlignment           97.5 +

In [58]:
records[0: 2]

[{'args': {'algorithm': 'HessianAlignment',
   'checkpoint_freq': None,
   'data_dir': './domainbed/data/',
   'dataset': 'VLCS',
   'device': 0,
   'holdout_fraction': 0.2,
   'hparams': '{"model_type":"ResNet"}',
   'hparams_seed': 1,
   'output_dir': './domainbed/results_resnet_new/be7d8e28947326fedd849898854fc595',
   'save_model_every_checkpoint': False,
   'seed': 728992854,
   'skip_model_save': False,
   'steps': None,
   'task': 'domain_generalization',
   'test_envs': [2],
   'trial_seed': 0,
   'uda_holdout_fraction': 0},
  'env0_in_acc': 0.6554770318021201,
  'env0_out_acc': 0.6784452296819788,
  'env1_in_acc': 0.49317647058823527,
  'env1_out_acc': 0.4858757062146893,
  'env2_in_acc': 0.49124143183549124,
  'env2_out_acc': 0.4817073170731707,
  'env3_in_acc': 0.4683450573861533,
  'env3_out_acc': 0.4725925925925926,
  'epoch': 0.0,
  'erm_loss': 1.5806922912597656,
  'grad_pen': 0.0,
  'hess_pen': 0.0,
  'hparams': {'batch_size': 39,
   'class_balanced': False,
   'data_au

In [12]:
input_dir

'./results_resnet'